<a href="https://colab.research.google.com/github/TinsaeTadesse17/Gaussian-splatting/blob/main/gsplat_colab_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Upload your dataset in Google Drive and mount in Colab. (If you want to train your own dataset)
1. Check if the selected GPU has CUDA.

In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
    print("CUDA version:", torch.version.cuda)
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")


CUDA is available.
CUDA version: 12.4
GPU: Tesla T4


2. Clone Gaussian Splatting repository.

In [2]:
%cd /content
!rm -rf gaussian-splatting
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

/content
Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 859, done.
remote: Total 859 (delta 0), reused 0 (delta 0), pack-reused 859 (from 1)
Receiving objects: 100% (859/859), 78.64 MiB | 38.60 MiB/s, done.
Resolving deltas: 100% (489/489), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core.git) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization.git) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim.git) registered for path 'submodules/fused-ssim'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3293, done.        
remote: Counting objects: 100% (322/322), done.        
remote: Compressing objects: 100%

3. Fetch sample data. Optional if you have your own dataset and mounted it via Drive. Change the name of /images folder to /input. Ex: .../db/playroom/images => .../db/playroom/input

In [3]:
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

--2025-08-05 14:30:47--  https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.49, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/69/11/6911555bcb5edae447d514169831329398b45a3ed9ae3b00817878bc501559a1/816e62f22a161abbfe841d2a6b10cdf036e297c9fa289b3bfeee9c6ec526d7e1?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tandt_db.zip%3B+filename%3D%22tandt_db.zip%22%3B&response-content-type=application%2Fzip&Expires=1754407847&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NDQwNzg0N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82OS8xMS82OTExNTU1YmNiNWVkYWU0NDdkNTE0MTY5ODMxMzI5Mzk4YjQ1YTNlZDlhZTNiMDA4MTc4NzhiYzUwMTU1OWExLzgxNmU2MmYyMmExNjFhYmJmZTg0MWQyYTZiMTBjZGYwMzZlMjk3YzlmYTI4OWIzYmZlZWU5YzZlYzUyN

4. Install dependencies for COLMAP:

In [4]:
!sudo apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-regex-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev \
    libsuitesparse-dev \
    libgflags-dev \
    libpng-dev \
    libjpeg-dev \
    libtiff-dev \
    libxxf86vm1 \
    libxxf86vm-dev \
    libxi-dev \
    libxrandr-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-filesystem-dev is already the newest version (1.74.0.3ubuntu7).
libboost-filesystem-dev set to manually installed.
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-regex-dev is already the newest version (1.74.0.3ubuntu7).
libboost-regex-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libjpeg-dev is already the newest version (8c-2ubuntu10).
libjpeg-dev set to manually installed.
libpng-dev is already the newest version (1.6.37-3build5).
libxxf86vm1 is already the newest version (1:1.1.4-1build3).
libxxf86vm1 set to manually installed.
libboost-graph-dev is already the newest version (1.74.0.3ubuntu7).
libboost-graph-dev set to manu

5. Build COLMAP. Takes ~30 minutes.

In [8]:
%cd /content
!git clone https://github.com/colmap/colmap.git
%cd colmap
!mkdir build
%cd build
!cmake ..
!make -j$(nproc)
!sudo make install

/content
fatal: destination path 'colmap' already exists and is not an empty directory.
/content/colmap
mkdir: cannot create directory ‘build’: File exists
/content/colmap/build
-- Enabling LSD support
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found FreeImage
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libfreeimage.so
-- Found Metis
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libmetis.so
-- Found Glog
--   Target : glog::glog
-- Found Glew
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libGLEW.so
-- Found required Ceres dependency: Eigen version 3.4.0 in /usr/include/eigen3
-- Found required Ceres dependency: glog
-- Found required Ceres dependency: gflags
-- Found Ceres version: 2.0.0 installed in: /usr with components: [EigenSparse, SparseLinearAlgebraLibrary, LAPACK, SuiteSparse, CXSparse, SchurSpecializations, Multithreading]
-- Using heade

7. Train the converted dataset.

In [28]:
# Replace {path_to_dataset}
# --save_iterations: arg for saved checkpoints. Set different number if you want.
!python train.py -s {path_to_dataset} --iterations 30000 --save_iterations 10000 20000 30000   #/content/gaussian-splatting/db/playroom

2025-08-05 15:10:51.606724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754406651.627223   16449 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754406651.633690   16449 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-05 15:10:51.654263: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Optimizing 
Output folder: ./output/9dce96ee-e [05/08 15:10:55]
Reading camera 225/225 [05/08 15:10:55]
Converting po

8. Download the .ply file in .../output/{sha}/point_cloud/{iteration_num}/point_cloud.py